### End to End demo of starting spot instance in AWS

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *

### Testing

In [3]:
list(ec2.key_pairs.all())

[ec2.KeyPairInfo(name='AmazonEC2test'),
 ec2.KeyPairInfo(name='aws-key-fast-ai'),
 ec2.KeyPairInfo(name='aws-key-test-vpc'),
 ec2.KeyPairInfo(name='id_rsa'),
 ec2.KeyPairInfo(name='Mac')]

#### Define parameters

In [4]:
vpc_name='fast-ai'

#### Get Existing VPC by tag name

In [5]:
vpc = get_vpc(vpc_name); vpc

ec2.Vpc(id='vpc-6e6b2a17')

#### Request Spot instance

In [7]:
instance_name = f'{vpc_name}-instance'
instance_type = 't2.micro'

In [8]:
spot_prices = get_spot_prices(); spot_prices[instance_type]

'0.008100'

In [10]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type).build()

In [16]:
launch_specs['BlockDeviceMappings'][0]['Ebs']['VolumeSize'] = 80

In [20]:
instance = create_spot_instance(instance_name, launch_specs, spot_price='0.5'); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-0f3b3ae1658268905
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@54.244.180.198


ec2.Instance(id='i-0f3b3ae1658268905')

Request on demand instance (if spot error)

In [ ]:
# instance = create_instance(instance_name, vpc, instance_type='t2.micro'); instance

In [ ]:
instance = get_instance(instance_name); instance

#### Attach EBS volume (Optional)

In [ ]:
volume_tag = 'fast-ai-ebs-volume-test'

In [ ]:
_ = attach_volume(instance, volume_tag, device='/dev/xvdf')

### SSH

In [ ]:
client = connect_to_instance(instance)

#### Mount EBS

In [ ]:
upload_path = Path.cwd()/'upload_scripts/mount_ebs.sh'
upload_file(client, str(upload_path), 'mount_ebs.sh')

In [ ]:
out, _ = run_command(client, 'chmod 755 mount_ebs.sh')

In [ ]:
out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf') # no reformatting
# out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf --reformat true')

In [ ]:
out, _ = run_command(client, 'ls ebs_mount_point') # no reformatting

#### Mount EFS

efs_addr = get_efs_address('fast-ai-efs'); efs_addr

In [ ]:
out, _ = run_command(client, 'mkdir ~/efs_mount_point')

In [ ]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [ ]:
out, _ = run_command(client, efs_mount_cmd)

In [ ]:
out, _ = run_command(client, 'ls efs_mount_point') # no reformatting

### TODO

* tmux
* Auto shutdown after training
* Convert to script